### Creación de un cargador de datos para NLP

Como ingeniero de IA trabajando en un proyecto de traducción de idiomas de vanguardia, tienes la tarea de cerrar la brecha de comunicación entre hablantes de diferentes lenguas. Traducir idiomas no es una tarea sencilla, especialmente considerando las complejidades, matices y contextos culturales que contienen. El éxito de este esfuerzo depende en gran medida de los datos: grandes corpus de frases bilingües que sirven como base para entrenar tus modelos.

En PyTorch, el cargador de datos juega un papel indispensable en la gestión de esta gran cantidad de datos. Para tareas de procesamiento de lenguaje natural (NLP) como la tuya, los datos suelen tener longitudes variables debido a las diferencias estructurales entre los idiomas. 
El cargador de datos agrupa eficientemente estas secuencias de longitud variable, garantizando que los modelos se entrenen con ejemplos diversos en cada iteración. Esta agrupación es crucial para aprovechar la computación paralela en las GPU, acelerando así el proceso de entrenamiento.

Además, el cargador de datos ayuda a mezclar aleatoriamente el conjunto de datos, lo que es esencial para evitar que los modelos memoricen la secuencia de datos de entrenamiento y fomentar una mejor generalización. Esto es particularmente importante en tareas de NLP, donde los datos pueden estar ordenados o agrupados por temas. La mezcla aleatoria asegura que el modelo siga siendo robusto y no desarrolle sesgos basados en el orden de los datos de entrada.

Por último, en el mundo del NLP, los pasos de preprocesamiento como la tokenización, el padding y la conversión numérica son fundamentales. El cargador de datos en PyTorch proporciona funciones que permiten integrar estos pasos de preprocesamiento de manera fluida, garantizando que los datos textuales en bruto se transformen en un formato adecuado para los modelos de aprendizaje profundo.

En este cuaderno, cubrirás todo el proceso de carga y procesamiento de datos de texto utilizando PyTorch.


### Configuración
#### Instalando librerías requeridas


In [ ]:
import sys
print(sys.executable)


#### Importando librerías requeridas


In [ ]:
import torchtext
print(torchtext.__version__)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random

#### **Conjunto de datos**

Un conjunto de datos en **PyTorch** es un objeto que representa una colección de muestras de datos. Cada muestra de datos típicamente consta de una o más características de entrada y sus correspondientes etiquetas objetivo. También puedes utilizar tu conjunto de datos para transformar tus datos según sea necesario.

#### **Cargador de datos**

Un cargador de datos en **PyTorch** es responsable de cargar y agrupar en lotes los datos de un conjunto de datos de manera eficiente. Abstrae el proceso de iterar sobre un conjunto de datos, mezclarlos y dividirlos en lotes para el entrenamiento. En aplicaciones de NLP, el cargador de datos se utiliza para procesar y transformar tus datos textuales, en lugar de simplemente utilizar el conjunto de datos.

Los cargadores de datos tienen varios parámetros clave, incluyendo el conjunto de datos desde el cual se cargan, el tamaño del lote (que determina cuántas muestras se incluyen por lote), si se mezclan los datos (shuffle, para cada época), y más. Los cargadores de datos también proporcionan una interfaz de iterador, lo que facilita iterar sobre los lotes de datos durante el entrenamiento.

#### '**¿Qué es un iterador?**'

Un iterador es un objeto sobre el cual se puede iterar. Contiene elementos que se pueden recorrer y típicamente incluye dos métodos, `__iter__()` y `__next__()`. Cuando no hay más elementos para iterar, lanza una excepción **`StopIteration`**.

Los iteradores se utilizan comúnmente para recorrer grandes conjuntos de datos sin cargar todos los elementos en memoria simultáneamente, haciendo el proceso más eficiente en cuanto a memoria. En PyTorch, no todos los conjuntos de datos son iteradores, pero todos los cargadores de datos lo son.

En PyTorch, el cargador de datos procesa la información en lotes, cargando y procesando un lote a la vez en memoria de manera eficiente. El tamaño del lote, que se especifica al crear el cargador de datos, determina cuántas muestras se procesan juntas en cada lote. El propósito del cargador de datos es convertir los datos de entrada y las etiquetas en lotes de tensores con la misma forma para que los modelos de aprendizaje profundo puedan interpretarlos.

Finalmente, un cargador de datos puede utilizarse para tareas como la tokenización, secuenciación, convertir tus muestras al mismo tamaño y transformar tus datos en tensores que tu modelo pueda entender.


#### **Conjunto de datos personalizado y cargador de datos en PyTorch**

En este fragmento de código, verás cómo crear un conjunto de datos personalizado y usar la clase DataLoader en PyTorch. El conjunto de datos consta de una lista de oraciones aleatorias, y el objetivo es crear lotes de oraciones para su posterior procesamiento, como el entrenamiento de un modelo de red neuronal.

Comenzarás definiendo un conjunto de datos personalizado llamado CustomDataset. Este conjunto de datos hereda de la clase `torch.utils.data.Dataset` y se inicializa con una lista de oraciones. El conjunto de datos comprende dos métodos esenciales:

- __init__(self, sentences): Inicializa el conjunto de datos con una lista de oraciones.
- __getitem__(self, idx): Recupera un elemento (en este caso, una oración) en un índice específico, idx.

A continuación, se crea una instancia de tu conjunto de datos personalizado (custom_dataset) pasando la lista de oraciones. Además, puedes especificar un tamaño de lote (batch_size), que determina cuántas oraciones se agruparán en cada lote durante la carga de datos.

Luego, crearás un DataLoader (dataloader) proporcionando tu conjunto de datos personalizado y el tamaño del lote a la clase `torch.utils.data.DataLoader`. Además, se establece `shuffle=True`, lo que indica que las oraciones se mezclarán aleatoriamente antes de dividirse en lotes. Esta mezcla es particularmente útil para entrenar modelos de aprendizaje profundo, ya que evita que el modelo aprenda patrones basados en el orden de los datos.

Finalmente, iterarás a través del DataLoader para demostrar cómo se cargan los datos en lotes. En este código, verás que el tamaño del lote se establece en 2, lo que significa que cada lote contendrá dos oraciones. El DataLoader gestiona eficientemente la carga de datos en lotes, lo que lo hace adecuado para entrenar modelos de aprendizaje profundo.

Durante la iteración, se imprimen las oraciones en cada lote para ilustrar cómo el DataLoader agrupa y presenta los datos. Este fragmento de código proporciona un ejemplo fundamental de cómo configurar un conjunto de datos personalizado y un cargador de datos en PyTorch, lo cual es una práctica común en los flujos de trabajo de aprendizaje profundo.

In [ ]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Definir un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences)

# Definir tamaño del lote
batch_size = 2

# Crear un DataLoader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterar a través del DataLoader
for batch in dataloader:
    print(batch)

Como se muestra arriba, los datos se organizan en lotes de 2 oraciones cada uno. Es importante notar que los modelos de aprendizaje profundo solo pueden comprender datos numéricos, y las palabras carecen de significado para ellos. Por lo tanto, tu siguiente paso es convertir estas oraciones en tensores. Veamos cómo hacerlo.

#### Creación de tensores para un conjunto de datos personalizado

En este ejemplo de código, verás la creación de un conjunto de datos personalizado para tareas de procesamiento de lenguaje natural (NLP) utilizando PyTorch. El conjunto de datos consiste en una lista de oraciones, y tu objetivo es preprocesar estas oraciones, tokenizarlas y convertirlas en tensores de índices de tokens para su uso en modelos de NLP. Desglosaremos el código paso a paso.

Las oraciones y la clase `CustomDataset` se utilizan de la misma manera que en el fragmento de código anterior. Los cambios realizados en la clase CustomDataset son los siguientes:

- __init__: El constructor recibe una lista de oraciones, una función de tokenización y un vocabulario (vocab) como entrada.
- __len__: Este método devuelve el número total de muestras en el conjunto de datos.
- __getitem__: Este método es responsable de procesar una muestra individual. Tokeniza la oración usando el tokenizador proporcionado y luego convierte los tokens en índices de tensores utilizando el vocabulario.

Puedes definir un tokenizador utilizando la función `get_tokenizer` con la opción `basic_english`. La tokenización es el proceso de dividir un texto en tokens o palabras individuales. A continuación, construyes un vocabulario a partir de las oraciones. Utilizas la función `build_vocab_from_iterator` para construir el vocabulario a partir de las oraciones tokenizadas.

Puedes crear una instancia de tu conjunto de datos personalizado, pasando las oraciones, el tokenizador y el vocabulario. Finalmente, imprimes la longitud del conjunto de datos personalizado y algunos elementos de muestra del conjunto de datos para ilustrarlo.


In [ ]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Definir un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx])
        # Convertir tokens a índices de tensor usando el vocabulario
        tensor_indices = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indices)

# Tokenizador
tokenizer = get_tokenizer("basic_english")

# Construir el vocabulario
vocab = build_vocab_from_iterator(map(tokenizer, sentences))

# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

print("Custom Dataset Length:", len(custom_dataset))
print("Sample Items:")
for i in range(6):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")

Adelante, descomenta el siguiente código para aplicar el cargador de datos y observar los resultados:

In [ ]:
"""
# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

# Definir tamaño del lote
batch_size = 2

# Crear un cargador de datos
#dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterar a través del cargador de datos
for batch in dataloader:
    print(batch)
"""

Encontrarás un error al intentar crear lotes para los tensores. Este error surge porque los lotes de tensores tienen longitudes desiguales. El cargador de datos está utilizando la función de colación (`collate_function`) por defecto, que requiere que los tensores tengan longitudes iguales. Puedes definir tu propia función de colación y pasarle los datos para establecer tus propias reglas. 

Típicamente, para resolver el problema de longitudes de tensor desiguales, se emplea el `padding` de datos. Esto se demostrará en la siguiente sección.

#### Función de colación personalizada

Una función de colación se emplea en el contexto de la carga de datos y el procesamiento por lotes en el aprendizaje automático, particularmente cuando se trata de datos de longitud variable, como secuencias (por ejemplo, texto, series temporales, secuencias de eventos). Su propósito principal es preparar y formatear muestras individuales de datos en lotes que puedan ser procesados eficientemente por los modelos de aprendizaje automático.

Comenzarás definiendo una función de colación personalizada llamada `collate_fn`. Esta función juega un papel crucial al manejar secuencias de longitudes variables, como oraciones en NLP. Su propósito es rellenar (pad) las secuencias dentro de un lote para que tengan longitudes iguales, lo cual es un paso de preprocesamiento común en tareas de NLP.

`pad_sequence`: Esta función forma parte de PyTorch y se utiliza para rellenar (pad) las secuencias en un lote, asegurando una longitud uniforme. Toma un lote de secuencias como entrada y las rellena para igualar la longitud de la secuencia más larga. El argumento `padding_value=0` especifica el valor a utilizar para el relleno.

In [ ]:
# Crear una función de colación personalizada
def collate_fn(batch):
    # Rellenar las secuencias dentro del lote para que tengan longitudes iguales
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return padded_batch

En la celda anterior, al rellenar las secuencias, se establece `batch_first=True`. Cuando `batch_first=True`, la salida tendrá la forma [tamaño_lote x longitud_secuencia], de lo contrario, tendrá la forma [longitud_secuencia x tamaño_lote]. Algunos modelos aceptan entradas con la forma [tamaño_lote x longitud_secuencia] mientras que otros necesitan que la entrada tenga la forma [longitud_secuencia x tamaño_lote]. Ten en cuenta que este parámetro se encarga de colocar la entrada en la forma deseada. 

Veamos cómo afecta realmente la forma de los lotes curados. Primero, creas un cargador de datos con una función de colación con `batch_first=True`:

In [ ]:
# Crear un cargador de datos con la función de colación personalizada con batch_first=True,
dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Iterar a través del cargador de datos
for batch in dataloader: 
    for row in batch:
        for idx in row:
            words = [vocab.get_itos()[idx] for idx in row]
        print(words)
       

Al observar el resultado, puedes ver que la primera dimensión es el lote. Por ejemplo, el primer lote es la primera oración: "['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']".

Ahora, puedes probar con `batch_first=False`, que es el valor por defecto:

In [ ]:
# Crear una función de colación personalizada
def collate_fn_bfFALSE(batch):
    # Rellenar las secuencias dentro del lote para que tengan longitudes iguales
    padded_batch = pad_sequence(batch, padding_value=0)
    return padded_batch

Ahora, observa los datos curados:

In [ ]:
# Crear un cargador de datos con la función de colación personalizada (batch_first=False)
dataloader_bfFALSE = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn_bfFALSE)

# Iterar a través del cargador de datos
for seq in dataloader_bfFALSE:
    for row in seq:
        #print(row)
        words = [vocab.get_itos()[idx] for idx in row]
        print(words)

Se puede observar que la primera dimensión es ahora la secuencia en lugar del lote, lo que significa que las oraciones se dividirán de manera que cada fila incluya un token de cada secuencia. Por ejemplo, la primera fila, (['if', 'fame']), incluye los primeros tokens de todas las secuencias en ese lote. Debes tener en cuenta este estándar para evitar confusiones al trabajar con redes neuronales recurrentes (RNNs) y transformers.

In [ ]:
# Iterar a través del cargador de datos con batch_first = TRUE
for batch in dataloader:    
    print(batch)
    print("Longitud de las secuencias en el lote:", batch.shape[1])

Verás que cada lote tiene un tamaño fijo para todas las secuencias dentro del lote.

También tienes la opción de utilizar la función de colación para tareas como la tokenización, la conversión de índices tokenizados y transformar el resultado en un tensor. Es importante tener en cuenta que el conjunto de datos original permanece sin modificaciones por estas transformaciones.

In [ ]:
# Definir un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [ ]:
custom_dataset = CustomDataset(sentences)

Tienes el texto sin procesar:

In [ ]:
custom_dataset[0]

Creas la nueva ```collate_fn```

In [ ]:
def collate_fn(batch):
    # Tokenizar cada muestra en el lote usando el tokenizador especificado
    tensor_batch = []
    for sample in batch:
        tokens = tokenizer(sample)
        # Convertir los tokens a índices del vocabulario y crear un tensor para cada muestra
        tensor_batch.append(torch.tensor([vocab[token] for token in tokens]))

    # Rellenar las secuencias dentro del lote para que tengan longitudes iguales usando pad_sequence
    # batch_first=True asegura que los tensores tengan la forma (tamaño_lote, longitud_secuencia_maxima)
    padded_batch = pad_sequence(tensor_batch, batch_first=True)
    
    # Devolver el lote rellenado
    return padded_batch

Crear un cargador de datos con la función de colación personalizada.

In [ ]:
# Crear un cargador de datos para el conjunto de datos personalizado
dataloader = DataLoader(
    dataset=custom_dataset,   # Conjunto de datos personalizado de PyTorch que contiene tus datos
    batch_size=batch_size,     # Número de muestras en cada mini-lote
    shuffle=True,              # Mezclar los datos al inicio de cada época
    collate_fn=collate_fn      # Función de colación personalizada para procesar los lotes
)

Verás que el resultado es un tensor con la misma forma para cada muestra en el lote.

In [ ]:
for batch in dataloader:
    print(batch)
    print("forma de la muestra", len(batch))

Como resultado, se han creado con éxito lotes de tensores con longitudes iguales.

#### Ejercicio


Crea un cargador de datos con una función de colación que procese lotes de texto en francés (proporcionado a continuación). Ordena el conjunto de datos según la longitud de las secuencias. Luego, tokeniza, numera y rellena las secuencias. Ordenar las secuencias minimizará la cantidad de tokens `<PAD>` añadidos a las secuencias, lo que mejora el rendimiento del modelo. Prepara los datos en lotes de tamaño 4 e imprímelos.

In [ ]:
corpus = [
    "Ceci est une phrase.",
    "C'est un autre exemple de phrase.",
    "Voici une troisième phrase.",
    "Il fait beau aujourd'hui.",
    "J'aime beaucoup la cuisine française.",
    "Quel est ton plat préféré ?",
    "Je t'adore.",
    "Bon appétit !",
    "Je suis en train d'apprendre le français.",
    "Nous devons partir tôt demain matin.",
    "Je suis heureux.",
    "Le film était vraiment captivant !",
    "Je suis là.",
    "Je ne sais pas.",
    "Je suis fatigué après une longue journée de travail.",
    "Est-ce que tu as des projets pour le week-end ?",
    "Je vais chez le médecin cet après-midi.",
    "La musique adoucit les mœurs.",
    "Je dois acheter du pain et du lait.",
    "Il y a beaucoup de monde dans cette ville.",
    "Merci beaucoup !",
    "Au revoir !",
    "Je suis ravi de vous rencontrer enfin !",
    "Les vacances sont toujours trop courtes.",
    "Je suis en retard.",
    "Félicitations pour ton nouveau travail !",
    "Je suis désolé, je ne peux pas venir à la réunion.",
    "À quelle heure est le prochain train ?",
    "Bonjour !",
    "C'est génial !"
]

In [ ]:
## Tu codigo de respuesta

#### [Opcional] Cargador de datos para la tarea de traducción Alemán-Inglés


Esta sección prepara el terreno para la traducción automática Alemán-Inglés utilizando las bibliotecas torchtext y spaCy. Ajusta las URLs del conjunto de datos Multi30k, configura los tokenizadores para ambos idiomas y establece vocabularios con tokens especiales. Esta base es crucial para construir y entrenar modelos de traducción efectivos.

- **Configuración del conjunto de datos y definición del idioma**
  - Se modifican las URLs por defecto del conjunto de datos Multi30k para corregir enlaces rotos.
  - Se definen los idiomas de origen (`de` para alemán) y destino (`en` para inglés).

- **Configuración del tokenizador**
  - Se configuran tokenizadores para ambos idiomas utilizando `spaCy`.

- **Generación de tokens**
  - Se crea una función auxiliar, `yield_tokens`, para generar tokens a partir del conjunto de datos.

- **Símbolos especiales**
  - Se definen los símbolos especiales (por ejemplo, `<unk>`, `<pad>`) y sus índices.

- **Construcción del vocabulario**
  - Se construyen vocabularios para ambos idiomas (origen y destino) a partir de los datos de **entrenamiento** del conjunto de datos Multi30k, convirtiendo tokens en índices únicos (números).

- **Manejo de tokens por defecto**
  - Se establece un índice por defecto (`UNK_IDX`) para los tokens que no se encuentren en el vocabulario.


#### Conjunto de datos de traducción
En esta sección, obtendrás un conjunto de datos de traducción llamado Multi30k. Modificarás las URLs de entrenamiento y validación por defecto, y luego recuperarás e imprimirás el primer par de oraciones en alemán e inglés del conjunto de entrenamiento. Primero, sobrescribirás las URLs por defecto:

In [ ]:
# Se modificarán las URLs del conjunto de datos ya que los enlaces al conjunto de datos original están rotos

multi30k.URL["train"] = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/training.tar.gz"
multi30k.URL["valid"] = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/validation.tar.gz"

Define el idioma de origen como Alemán ('de') y el idioma de destino como Inglés ('en'). En Python, las variables globales son aquellas definidas fuera de una función, accesibles tanto dentro como fuera de las funciones. A menudo se escriben en mayúsculas como una convención para indicar que son constantes, de naturaleza global y para diferenciarlas de las variables normales.

In [ ]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

Inicializa el iterador de datos de entrenamiento para el conjunto de datos Multi30k con los idiomas de origen y destino especificados:

In [ ]:
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

Crea un iterador para el conjunto de datos de entrenamiento:

In [ ]:
data_set = iter(train_iter)

Puedes imprimir los primeros cinco pares de oraciones de origen y destino del conjunto de datos de entrenamiento:

In [ ]:
for n in range(5):
    # Obtener el siguiente par de oraciones de origen y destino del conjunto de datos de entrenamiento
    src, tgt = next(data_set)

    # Imprimir las oraciones de origen (alemán) y destino (inglés)
    print(f"muestra {str(n+1)}")
    print(f"Origen ({SRC_LANGUAGE}): {src}\nDestino ({TGT_LANGUAGE}): {tgt}")

#### Configuración del tokenizador

El tokenizador, configurado usando spaCy, descompone el texto en unidades más pequeñas o tokens, facilitando un procesamiento lingüístico preciso y asegurando que las palabras y puntuaciones se segmenten adecuadamente para la tarea de traducción. Utilicemos los siguientes ejemplos:

In [ ]:
german, english = next(data_set)
print(f"Origen Alemán ({SRC_LANGUAGE}): {german}\nDestino Inglés ({TGT_LANGUAGE}): { english }")

Importa la función utilitaria ```get_tokenizer``` de ```torchtext``` para obtener tokenizadores para el procesamiento del lenguaje:

In [ ]:
from torchtext.data.utils import get_tokenizer

Inicializa los tokenizadores para alemán e inglés utilizando el modelo 'de_core_news_sm' de spaCy:

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


In [ ]:
# Crear un diccionario para almacenar ambos tokenizadores
token_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

La línea ```token_transform['de'](german)``` tokenizará la cadena en alemán (o texto) utilizando el tokenizador previamente definido ```token_transform['de']``` para el idioma alemán.

In [ ]:
token_transform['de'](german)

Lo mismo para el inglés:

In [ ]:
token_transform['en'](english)

#### Símbolos especiales
En un contexto típico de NLP, los tokens `['<unk>', '<pad>', '<bos>', '<eos>']` tienen significados específicos:

1. `<unk>`: Este token representa "desconocido" o "fuera del vocabulario". Se utiliza cuando una palabra en el texto de entrada no se encuentra en el vocabulario o cuando se trata de palabras raras o no vistas durante el entrenamiento. Cuando el modelo se encuentra con una palabra desconocida, la reemplaza con el token `<unk>`.

2. `<pad>`: Este token representa el relleno. En secuencias de datos textuales, como oraciones o documentos, las secuencias pueden tener diferentes longitudes. Para crear lotes de datos con dimensiones uniformes, las secuencias más cortas a menudo se rellenan con este token `<pad>` para igualar la longitud de la secuencia más larga en el lote.

3. `<bos>`: Este token representa el "inicio de secuencia." Se utiliza para indicar el comienzo de una oración o secuencia de tokens. Ayuda al modelo a entender el inicio de una secuencia de texto.

4. `<eos>`: Este token representa el "fin de secuencia." Se utiliza para indicar el final de una oración o secuencia de tokens. Ayuda al modelo a reconocer el final de una secuencia de texto.

Definir símbolos especiales e índices


In [ ]:
# Definir símbolos especiales e índices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Asegurarse de que los tokens estén en orden de sus índices para insertarlos correctamente en el vocabulario
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

#### Transformación de tokens a índices (Vocabulario)
El código inicializa un diccionario vocab_transform y luego construye vocabularios para ambos idiomas, alemán (de) e inglés (en), a partir del conjunto de datos ```train_iter``` utilizando la función auxiliar ```yield_tokens```. Estos vocabularios se almacenan en el diccionario vocab_transform. Los vocabularios se construyen con ciertas restricciones, como una frecuencia mínima para los tokens y la inclusión de símbolos especiales al inicio.

Inicializa un diccionario para almacenar los vocabularios de ambos idiomas:

In [ ]:
# Diccionario de reserva para las transformaciones de vocabulario de 'en' y 'de'
vocab_transform = {}

Crearás una función `yield_tokens` que procese un iterador de conjunto de datos dado (`data_iter`) y, para cada muestra, tokenice los datos para el idioma especificado (language). Utiliza un mapeo predefinido (`token_transform`) de idiomas a sus respectivos tokenizadores.

In [ ]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    # Definir un mapeo para asociar los idiomas de origen y destino con sus respectivas posiciones en las muestras de datos.
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    # Iterar sobre cada muestra de datos en el iterador del conjunto de datos proporcionado
    for data_sample in data_iter:
        # Tokenizar la muestra de datos correspondiente al idioma especificado y devolver los tokens resultantes.
        yield token_transform[language](data_sample[language_index[language]])


Construyes y almacenas los vocabularios de alemán e inglés únicamente a partir del conjunto de datos de **entrenamiento**. Puedes utilizar la función auxiliar ```yield_tokens``` para tokenizar los datos. Incluye tokens que aparezcan al menos una vez (`min_freq=1`) y añade símbolos especiales (como <pad>, <unk>, etc.) al inicio del vocabulario:

In [ ]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Iterador de datos de entrenamiento
    train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Para disminuir la cantidad de tokens de relleno, ordenas los datos según la longitud de la fuente para agrupar secuencias de longitudes similares
    sorted_dataset = sorted(train_iterator, key=lambda x: len(x[0].split()))
    # Crear el objeto Vocab de torchtext
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(sorted_dataset, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

Establece ```UNK_IDX``` como el índice por defecto. Este índice se devuelve cuando el token no se encuentra.

In [ ]:
# Si no se establece, lanza un ``RuntimeError`` cuando el token consultado no se encuentra en el vocabulario.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

Tomarás la cadena de texto en inglés/alemán, la tokenizarás en palabras o subpalabras, y luego convertirás estos tokens en sus índices correspondientes del vocabulario, resultando en una secuencia de enteros, `seq_en`, que se puede utilizar para un procesamiento posterior en un modelo.

In [ ]:
seq_en = vocab_transform['en'](token_transform['en'](english))
print(f"Cadena de texto en inglés: {english}\nSecuencia en inglés: {seq_en}")

seq_de = vocab_transform['de'](token_transform['de'](german))
print(f"Cadena de texto en alemán: {german}\nSecuencia en alemán: {seq_de}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

La función ```tensor_transform_s``` añade un token de inicio de secuencia (BOS) al comienzo, invierte la secuencia para revertir el orden de los IDs de tokens y añade un token de fin de secuencia (EOS) al final de una secuencia dada de IDs de tokens, luego devuelve el resultado concatenado como un tensor de PyTorch; esto se utilizará como entrada para nuestro modelo.

La función ```tensor_transform_t``` realiza operaciones similares, excepto la inversión. Es una buena práctica invertir el orden de la oración de origen para que el LSTM funcione mejor.

In [ ]:
# función para añadir BOS/EOS, invertir la oración de origen y crear un tensor para los índices de la secuencia de entrada
def tensor_transform_s(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.flip(torch.tensor(token_ids), dims=(0,)),
                      torch.tensor([EOS_IDX])))

# función para añadir BOS/EOS y crear un tensor para los índices de la secuencia de entrada
def tensor_transform_t(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

In [ ]:
seq_en = tensor_transform_s(seq_en)
seq_en

In [ ]:
seq_de = tensor_transform_t(seq_de)
seq_de

Ahora que has definido la función de transformación, creas una función ```sequential_transforms``` para agrupar todas las transformaciones en el orden correcto.


In [ ]:
# función auxiliar para agrupar operaciones secuenciales
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# Transformaciones de texto para los idiomas ``src`` y ``tgt`` para convertir cadenas sin procesar en índices de tensores
text_transform = {}

text_transform[SRC_LANGUAGE] = sequential_transforms(token_transform[SRC_LANGUAGE],  # Tokenización
                                            vocab_transform[SRC_LANGUAGE],         # Numerización
                                            tensor_transform_s)                    # Añadir BOS/EOS y crear tensor

text_transform[TGT_LANGUAGE] = sequential_transforms(token_transform[TGT_LANGUAGE],  # Tokenización
                                            vocab_transform[TGT_LANGUAGE],         # Numerización
                                            tensor_transform_t)                    # Añadir BOS/EOS y crear tensor

#### Procesamiento de datos en lotes
La función collate_fn se basa en las utilidades que estableciste anteriormente. Aplica la transformación de texto (text_transform) a un lote de datos sin procesar. Además, asegura longitudes de secuencia consistentes dentro del lote mediante el relleno. Esta transformación prepara los datos para ser utilizados como entrada en un modelo transformer diseñado para tareas de traducción de idiomas.

In [ ]:
# función para agrupar muestras de datos en tensores por lotes
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_sequences = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        #src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
        src_sequences = src_sequences.clone().detach().to(torch.int64)
        tgt_sequences = text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n"))
        #tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
        tgt_sequences = tgt_sequences.clone().detach().to(torch.int64)
        src_batch.append(src_sequences)
        tgt_batch.append(tgt_sequences)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    
    return src_batch.to(device), tgt_batch.to(device)

Estableces un iterador de datos de entrenamiento utilizando el conjunto de datos Multi30k y configuras un cargador de datos con un tamaño de lote de 4. Esto aprovecha la función collate_fn predefinida para curar y preparar eficientemente los lotes para entrenar tu modelo transformer. Tu objetivo principal es profundizar en las complejidades de los componentes del codificador y decodificador de la RNN.

In [ ]:
BATCH_SIZE = 4

train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
sorted_train_iterator = sorted(train_iterator, key=lambda x: len(x[0].split()))
train_dataloader = DataLoader(sorted_train_iterator, batch_size=BATCH_SIZE, collate_fn=collate_fn, drop_last=True)

valid_iterator = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
sorted_valid_dataloader = sorted(valid_iterator, key=lambda x: len(x[0].split()))
valid_dataloader = DataLoader(sorted_valid_dataloader, batch_size=BATCH_SIZE, collate_fn=collate_fn, drop_last=True)


src, trg = next(iter(train_dataloader))
src, trg